In [5]:
import pymysql
import configparser
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from itertools import combinations

from tqdm.notebook import tqdm

In [2]:
config = configparser.ConfigParser()
filename = './resource/db.ini'
config.read(filename)

['./resource/db.ini']

In [3]:
connection = pymysql.connect(host=config['DATABASE']['HOST'],
                             user=config['DATABASE']['USERNAME'],
                             password=config['DATABASE']['PASSWORD'],
                             db="patent_20200630",
                             charset='utf8mb4',
                             defer_connect=True)

In [4]:
if not connection.open:
    connection.connect()
with connection.cursor() as cursor:
    cursor.execute("Select 1")

In [5]:
def get_dataframe_from_pymysql_cursor(connection, query):
    if not connection.open:
        connection.connect()
    with connection.cursor() as cursor:
        cursor.execute(query)
        results = pd.DataFrame(cursor.fetchall(),
                               columns=[i[0] for i in cursor.description])
    return results

# Canonical Name Assignment

In [ ]:
limit = 10000
offset=0

In [ ]:
new_data_id_query_template = """
SELECT disambiguated_id 
from tmp_inventor_disambiguation_granted_ids order by disambiguated_id limit {l} offset {o}
"""

new_data_query_template = """
SELECT tidg.disambiguated_id , r.name_first, r.name_last from tmp_inventor_disambiguation_granted tidg
         join ({id_query}) tidg2
              on tidg2.disambiguated_id = tidg.disambiguated_id
         join rawinventor r on tidg.uuid = r.uuid order by tidg.disambiguated_id
"""

In [ ]:
pbar = tqdm(total=17987248)
mode='w'
while True:
    id_query = new_data_id_query_template.format(l=limit, o=offset)
    data_query = new_data_query_template.format(id_query=id_query)
    current_df = get_dataframe_from_pymysql_cursor(connection, data_query)
    if current_df.shape[0] < 1:
        break
    count_frame = current_df.groupby(
        ["disambiguated_id", "name_first",
         "name_last"]).agg(len).reset_index().rename({0: "count"}, axis=1)

    inventors_data = count_frame.sort_values(
        "count",
        ascending=False).groupby("disambiguated_id").head(1).reset_index(
            drop=True).drop("count", axis=1)
    inventors_data.to_csv("./data/tmp_inventor_disambig_canonical_names.csv",
                          index=False,
                          mode=mode)
    mode='a'
    pbar.update(limit)
    offset += limit
pbar.close()

In [ ]:
old_data_cluster_names = get_dataframe_from_pymysql_cursor(
    connection, "SELECT id, name_first, name_last from inventor")

In [ ]:
def calculate_cluster_distances(cluster_data):
    distance_map = {
        "levenshtein": distance,
        'jaro': jaro,
        'jaro_winkler': jaro_winkler
    }
    current_cluster_stats = {
        'N': cluster_data.shape[0],
        'levenshtein': 0,
        'jaro': 0,
        'jaro_winkler': 0
    }
    if cluster_data.shape[0] == 1:
        current_cluster_stats['levenshtein'] = None
        current_cluster_stats['jaro'] = None
        current_cluster_stats['jaro_winkler'] = None
    elif cluster_data.shape[0] > 0:
        for x in combinations(cluster_data.inventor_name, 2):
            for distance_method, distance_function in distance_map.items():
                #                 print("DIstance between {f} and {s} under {m} is {d}".format(
                #                     f=x[0],
                #                     s=x[1],
                #                     m=distance_method,
                #                     d=distance_function(x[0], x[1])))
                current_cluster_stats[distance_method] += distance_function(
                    x[0], x[1])

    return (pd.Series(current_cluster_stats))

In [ ]:


old_data_cluster_names.to_csv("./data/inventor_disambig_canonical_names.csv", index=False)

In [3]:
from itertools import combinations

In [6]:
new_data=pd.read_csv("./data/tmp_inventor_disambig_canonical_names.csv")
new_data.set_index("disambiguated_id", inplace=True)

In [14]:
new_data.shape


(3455267, 2)

In [ ]:
mode='w'
for c in tqdm(combinations(new_data.index, 2), total=5969433293011):
    rows = pd.concat([new_data.loc[c[0]].reset_index(), new_data.loc[c[1]].reset_index()],keys=['x','y'], axis=1)
    rows.columns=["_".join(x) for x in rows.columns]
    rows.to_csv("./data/new_disambig_combinations.csv", index=False, mode=mode)

In [ ]:
for x, y in zip(*combinations(new_data.index, 2)):
    print(x)
    print(y)
    break

In [ ]:
inventor_combination = pd.concat(
    [new_data.loc[x].reset_index(), new_data.loc[y].reset_index()],
    
)